In [96]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import re
import csv

In [3]:
import urllib.request

# get list of negative and positive words
#negative
with urllib.request.urlopen('http://www.unc.edu/~ncaren/haphazard/negative.txt') as response:
    bop = response.read()

negative_words = str(bop)
# randomly python doesn't recognize \ on my mac, anyways
negative_words = negative_words.replace('\\n', ' ')
negative_words = negative_words.split(' ')
    
#positive
with urllib.request.urlopen('http://www.unc.edu/~ncaren/haphazard/positive.txt') as response:
    bop = response.read()

positive_words = str(bop)
# randomly python doesn't recognize \ on my mac, anyways
positive_words = positive_words.replace('\\n', ' ')
positive_words = positive_words.split(' ')

In [7]:
# define all the words we don't want when we do the text analysis
# using a standard list of "stop words" I got from somewhere...

unwanted_words = open('/Users/MatthewCourchene/Downloads/terrierstop.txt').read()

#import csv
# apparently need the full filepath here (~ wasn't working)
#with open('/Users/MatthewCourchene/Downloads/terrierstop.txt', newline='') as csvfile:
#    spamreader = csv.reader(csvfile, delimiter=' ', quotechar='|')
#    for row in spamreader:
#        unwanted_words.append(','.join(row))

more_unwanted_words = ['just', 'know', 'going', 'golf', 'like', 'thats', 'here', 'ball',
                      'know', 'course', 'think', 'dont', 'yeah']


unwanted_words = unwanted_words.split()
unwanted_words = unwanted_words + more_unwanted_words


In [103]:
from string import punctuation

# first, define function to count unique words in a string

def count_unique_words(my_string):
    
    # function that takes a string of words and counts the number 
    # of times each unique word appears
    
    no_punc = ""
    # get rid of punctuation
    for i in my_string:
        if i not in punctuation:
            no_punc = no_punc + i
        else:
            no_punc = no_punc + ""
        
    # make a list with each element as a word from string
    # get everything lower case
    words = str.lower(no_punc).split()
    
    # get rid of stop words defined previously as "unwanted_words"
    words = [word for word in words if word not in unwanted_words]
    words = [word for word in words if len(word) > 3]
    # get list of unique words
    unique_words = []
    for i in words:
        if i not in unique_words:
            unique_words.append(i)

    # count number of times each unique word appears
    word_counter = []
    for i in unique_words:
        count = 0
        for j in words:
            if i == j:
                count += 1
        word_counter.append(count)

    # make DataFrame
    data =  np.column_stack((unique_words, word_counter))
    data2 = pd.DataFrame(data=data, columns = ['word', 'count'])
    return(data2)

# second, define function to calculate # of positive and negative words as a fraction of all words

def sentiment_analysis(my_string):
    no_punc = ""
    # get rid of punctuation
    for i in my_string:
        if i not in punctuation:
            no_punc = no_punc + i
        else:
            no_punc = no_punc + ""
        
    # make a list with each element as a word from string
    # get everything lower case
    words = str.lower(no_punc).split()
    
    positive_counter = 0
    negative_counter = 0
    for i in words:
        if i in positive_words:
            #print('positive word:' + i)
            positive_counter += 1
        if i in negative_words:
            #print('negative word:' + i)
            negative_counter += 1
        else:
            continue
            
    negative_fraction = negative_counter/len(words)
    positive_fraction = positive_counter/len(words)
    
    return(negative_fraction, positive_fraction, len(words))
            
        
        
    
    

In [105]:
r_main = requests.get('http://www.asapsports.com/show_player.php?id=10085')
main_soup = BeautifulSoup(r_main.content, 'html.parser')

#final_data = pd.DataFrame()
all_data = []

for interview in main_soup.findAll('table')[-2].findAll('tr'):
    date = interview.find('nobr').get_text().replace('[ ', '').replace(' ]', '')
    event = interview.find('a').get_text()
    link = interview.find('a')['href']
    print('fetching: ' + date + ' - ' + event + '...')
    
    # use the link to scrape his interview
    r_sub = requests.get(link)
    sub_soup = BeautifulSoup(r_sub.content, 'html.parser')
    # loop through all elements that contain text: 'TIGER WOODS:'
    all_words = ''
    for answer in sub_soup.findAll(text = re.compile('TIGER WOODS')):
        text = answer.lstrip().replace('TIGER WOODS:','').replace(',','').replace('.', ' ')
        all_words += text
    
    # run function to clean text and count unique words
    # another function calculates fraction of positive and negative words
    # data = count_unique_words(all_words)
    new = {}
    new['date'] = date
    new['event'] = event
    new['positive'] = sentiment_analysis(all_words)[1]
    new['negative'] = sentiment_analysis(all_words)[0]
    new['word_count'] = sentiment_analysis(all_words)[2]
    # finally append this dictionary to the grand list
    all_data.append(new)
    
    # add columns to data frame
    #data['date'] = date
    #data['event'] = event
    
    #final_data = final_data.append(data)
    
print('Done!')

#print(final_data)
#write to csv
#final_data.to_csv('~/Dropbox/tiger_interviews', index = False)
all_data = pd.DataFrame(all_data)
#print(all_data)
all_data.to_csv('~/Desktop/tiger_sentiments', index = False)

fetching: February 2, 2017 - OMEGA DUBAI DESERT CLASSIC...
fetching: February 1, 2017 - OMEGA DUBAI DESERT CLASSIC...
fetching: October 2, 2016 - THE RYDER CUP...
fetching: August 23, 2015 - WYNDHAM CHAMPIONSHIP...
fetching: August 22, 2015 - WYNDHAM CHAMPIONSHIP...
fetching: August 21, 2015 - WYNDHAM CHAMPIONSHIP...
fetching: August 20, 2015 - WYNDHAM CHAMPIONSHIP...
fetching: August 19, 2015 - WYNDHAM CHAMPIONSHIP...
fetching: August 15, 2015 - PGA CHAMPIONSHIP...
fetching: August 13, 2015 - PGA CHAMPIONSHIP...
fetching: August 11, 2015 - PGA CHAMPIONSHIP...
fetching: August 3, 2015 - DEUTSCHE BANK CHAMPIONSHIP MEDIA DAY...
fetching: August 2, 2015 - QUICKEN LOANS NATIONAL...
fetching: August 1, 2015 - QUICKEN LOANS NATIONAL...
fetching: July 31, 2015 - QUICKEN LOANS NATIONAL...
fetching: July 30, 2015 - QUICKEN LOANS NATIONAL...
fetching: July 28, 2015 - QUICKEN LOANS NATIONAL...
fetching: July 18, 2015 - THE OPEN CHAMPIONSHIP...
fetching: July 16, 2015 - THE OPEN CHAMPIONSHIP...
fe

fetching: July 21, 2013 - THE OPEN CHAMPIONSHIP...
fetching: July 20, 2013 - THE OPEN CHAMPIONSHIP...
fetching: July 19, 2013 - THE OPEN CHAMPIONSHIP...
fetching: July 18, 2013 - THE OPEN CHAMPIONSHIP...
fetching: July 16, 2013 - THE OPEN CHAMPIONSHIP...
fetching: July 16, 2013 - THE OPEN CHAMPIONSHIP...
fetching: June 30, 2013 - AT&T; NATIONAL...
fetching: June 26, 2013 - AT&T; NATIONAL...
fetching: June 16, 2013 - U.S. OPEN CHAMPIONSHIP...
fetching: June 15, 2013 - U.S. OPEN CHAMPIONSHIP...
fetching: June 14, 2013 - U.S. OPEN CHAMPIONSHIP...
fetching: June 14, 2013 - U.S. OPEN CHAMPIONSHIP...
fetching: June 13, 2013 - U.S. OPEN CHAMPIONSHIP...
fetching: June 11, 2013 - U.S. OPEN CHAMPIONSHIP...
fetching: June 2, 2013 - THE MEMORIAL TOURNAMENT PRESENTED BY NATIONWIDE INSURANCE...
fetching: June 1, 2013 - THE MEMORIAL TOURNAMENT PRESENTED BY NATIONWIDE INSURANCE...
fetching: May 31, 2013 - THE MEMORIAL TOURNAMENT PRESENTED BY NATIONWIDE INSURANCE...
fetching: May 30, 2013 - THE MEMORIA

fetching: May 4, 2012 - WELLS FARGO CHAMPIONSHIP...
fetching: May 3, 2012 - WELLS FARGO CHAMPIONSHIP...
fetching: April 8, 2012 - MASTERS TOURNAMENT...
fetching: April 7, 2012 - MASTERS TOURNAMENT...
fetching: April 6, 2012 - MASTERS TOURNAMENT...
fetching: April 6, 2012 - MASTERS TOURNAMENT...
fetching: April 5, 2012 - MASTERS TOURNAMENT...
fetching: April 5, 2012 - MASTERS TOURNAMENT...
fetching: April 3, 2012 - MASTERS TOURNAMENT...
fetching: March 25, 2012 - ARNOLD PALMER INVITATIONAL PRESENTED BY MASTERCARD...
fetching: March 25, 2012 - ARNOLD PALMER INVITATIONAL PRESENTED BY MASTERCARD...
fetching: March 24, 2012 - ARNOLD PALMER INVITATIONAL PRESENTED BY MASTERCARD...
fetching: March 23, 2012 - ARNOLD PALMER INVITATIONAL PRESENTED BY MASTERCARD...
fetching: March 22, 2012 - ARNOLD PALMER INVITATIONAL PRESENTED BY MASTERCARD...
fetching: March 21, 2012 - ARNOLD PALMER INVITATIONAL PRESENTED BY MASTERCARD...
fetching: March 10, 2012 - WGC CADILLAC CHAMPIONSHIP...
fetching: March 9,

fetching: August 10, 2010 - PGA CHAMPIONSHIP...
fetching: August 8, 2010 - WGC BRIDGESTONE INVITATIONAL...
fetching: August 7, 2010 - WGC BRIDGESTONE INVITATIONAL...
fetching: August 5, 2010 - WGC BRIDGESTONE INVITATIONAL...
fetching: August 4, 2010 - WGC BRIDGESTONE INVITATIONAL...
fetching: July 18, 2010 - THE OPEN CHAMPIONSHIP...
fetching: July 17, 2010 - THE OPEN CHAMPIONSHIP...
fetching: July 16, 2010 - THE OPEN CHAMPIONSHIP...
fetching: July 15, 2010 - THE OPEN CHAMPIONSHIP...
fetching: July 13, 2010 - THE OPEN CHAMPIONSHIP...
fetching: July 6, 2010 - JP McMANUS INVITATIONAL PRO-AM...
fetching: July 4, 2010 - AT&T; NATIONAL...
fetching: July 3, 2010 - AT&T; NATIONAL...
fetching: July 2, 2010 - AT&T; NATIONAL...
fetching: July 1, 2010 - AT&T; NATIONAL...
fetching: June 29, 2010 - AT&T; NATIONAL...
fetching: June 20, 2010 - U.S. OPEN CHAMPIONSHIP...
fetching: June 19, 2010 - U.S. OPEN CHAMPIONSHIP...
fetching: June 19, 2010 - U.S. OPEN CHAMPIONSHIP...
fetching: June 19, 2010 - U.S.

fetching: June 15, 2008 - U.S. OPEN CHAMPIONSHIP...
fetching: June 14, 2008 - U.S. OPEN CHAMPIONSHIP...
fetching: June 14, 2008 - U.S. OPEN CHAMPIONSHIP...
fetching: June 13, 2008 - U.S. OPEN CHAMPIONSHIP...
fetching: June 13, 2008 - U.S. OPEN CHAMPIONSHIP...
fetching: June 12, 2008 - U.S. OPEN CHAMPIONSHIP...
fetching: June 10, 2008 - U.S. OPEN CHAMPIONSHIP...
fetching: June 2, 2008 - PGA CHAMPIONSHIP MEDIA DAY...
fetching: May 27, 2008 - AT&T; NATIONAL MEDIA DAY...
fetching: May 12, 2008 - BMW CHAMPIONSHIP MEDIA DAY...
fetching: April 13, 2008 - MASTERS TOURNAMENT...
fetching: April 13, 2008 - MASTERS TOURNAMENT...
fetching: April 12, 2008 - MASTERS TOURNAMENT...
fetching: April 12, 2008 - MASTERS TOURNAMENT...
fetching: April 11, 2008 - MASTERS TOURNAMENT...
fetching: April 11, 2008 - MASTERS TOURNAMENT...
fetching: April 10, 2008 - MASTERS TOURNAMENT...
fetching: April 10, 2008 - MASTERS TOURNAMENT...
fetching: April 8, 2008 - MASTERS TOURNAMENT...
fetching: March 24, 2008 - WGC CA

...

fetching: August 18, 2005 - WGC NEC INVITATIONAL...
fetching: August 17, 2005 - WGC NEC INVITATIONAL...
fetching: August 14, 2005 - PGA CHAMPIONSHIP...
fetching: August 13, 2005 - PGA CHAMPIONSHIP...
fetching: August 12, 2005 - PGA CHAMPIONSHIP...
fetching: August 11, 2005 - PGA CHAMPIONSHIP...
fetching: August 9, 2005 - PGA CHAMPIONSHIP...
fetching: July 31, 2005 - BUICK OPEN...
fetching: July 30, 2005 - BUICK OPEN...
fetching: July 29, 2005 - BUICK OPEN...
fetching: July 28, 2005 - BUICK OPEN...
fetching: July 27, 2005 - BUICK OPEN...
fetching: July 17, 2005 - THE OPEN CHAMPIONSHIP...
fetching: July 16, 2005 - THE OPEN CHAMPIONSHIP...
fetching: July 15, 2005 - THE OPEN CHAMPIONSHIP...
fetching: July 14, 2005 - THE OPEN CHAMPIONSHIP...
fetching: July 12, 2005 - THE OPEN CHAMPIONSHIP...
fetching: July 3, 2005 - CIALIS WESTERN OPEN...
fetching: July 2, 2005 - CIALIS WESTERN OPEN...
fetching: July 1, 2005 - CIALIS WESTERN OPEN...
fetching: June 30, 2005 - CIALIS WESTERN OPEN...
fetching:

fetching: March 20, 2004 - BAY HILL INVITATIONAL PRESENTED BY MASTERCARD...
fetching: March 19, 2004 - BAY HILL INVITATIONAL PRESENTED BY MASTERCARD...
fetching: March 18, 2004 - BAY HILL INVITATIONAL PRESENTED BY MASTERCARD...
fetching: March 16, 2004 - BAY HILL INVITATIONAL PRESENTED BY MASTERCARD...
fetching: March 7, 2004 - DUBAI DESERT CLASSIC...
fetching: March 6, 2004 - DUBAI DESERT CLASSIC...
fetching: March 5, 2004 - DUBAI DESERT CLASSIC...
fetching: March 3, 2004 - DUBAI DESERT CLASSIC...
fetching: February 29, 2004 - WGC ACCENTURE MATCH PLAY CHAMPIONSHIP...
fetching: February 28, 2004 - WGC ACCENTURE MATCH PLAY CHAMPIONSHIP...
fetching: February 27, 2004 - WGC ACCENTURE MATCH PLAY CHAMPIONSHIP...
fetching: February 27, 2004 - WGC ACCENTURE MATCH PLAY CHAMPIONSHIP...
fetching: February 25, 2004 - WGC ACCENTURE MATCH PLAY CHAMPIONSHIP...
fetching: February 24, 2004 - WGC ACCENTURE MATCH PLAY CHAMPIONSHIP...
fetching: February 15, 2004 - BUICK INVITATIONAL...
fetching: February

fetching: May 9, 2002 - VERIZON BYRON NELSON CLASSIC...
fetching: May 8, 2002 - VERIZON BYRON NELSON CLASSIC...
fetching: April 14, 2002 - MASTERS TOURNAMENT...
fetching: April 13, 2002 - MASTERS TOURNAMENT...
fetching: April 11, 2002 - MASTERS TOURNAMENT...
fetching: April 9, 2002 - MASTERS TOURNAMENT...
fetching: March 24, 2002 - THE PLAYERS CHAMPIONSHIP...
fetching: March 22, 2002 - THE PLAYERS CHAMPIONSHIP...
fetching: March 21, 2002 - THE PLAYERS CHAMPIONSHIP...
fetching: March 20, 2002 - THE PLAYERS CHAMPIONSHIP...
fetching: March 17, 2002 - BAY HILL INVITATIONAL PRESENTED BY COOPER TIRES...
fetching: March 16, 2002 - BAY HILL INVITATIONAL PRESENTED BY COOPER TIRES...
fetching: March 15, 2002 - BAY HILL INVITATIONAL PRESENTED BY COOPER TIRES...
fetching: March 14, 2002 - BAY HILL INVITATIONAL PRESENTED BY COOPER TIRES...
fetching: March 12, 2002 - BAY HILL INVITATIONAL PRESENTED BY COOPER TIRES...
fetching: March 4, 2002 - PGA TOUR MEDIA CONFERENCE...
fetching: March 3, 2002 - GE

fetching: November 4, 1999 - WGC AMERICAN EXPRESS CHAMPIONSHIP...
fetching: November 2, 1999 - WGC AMERICAN EXPRESS CHAMPIONSHIP...
fetching: October 31, 1999 - THE TOUR CHAMPIONSHIP PRESENTED BY SOUTHERN COMPANY...
fetching: October 30, 1999 - THE TOUR CHAMPIONSHIP PRESENTED BY SOUTHERN COMPANY...
fetching: October 28, 1999 - THE TOUR CHAMPIONSHIP PRESENTED BY SOUTHERN COMPANY...
fetching: October 26, 1999 - THE TOUR CHAMPIONSHIP PRESENTED BY SOUTHERN COMPANY...
fetching: August 29, 1999 - WGC NEC INVITATIONAL...
fetching: August 28, 1999 - WGC NEC INVITATIONAL...
fetching: August 25, 1999 - WGC NEC INVITATIONAL...
fetching: August 18, 1999 - SPRINT INTERNATIONAL...
fetching: June 20, 1999 - U.S. OPEN CHAMPIONSHIP...
fetching: June 19, 1999 - U.S. OPEN CHAMPIONSHIP...
fetching: June 18, 1999 - U.S. OPEN CHAMPIONSHIP...
fetching: June 17, 1999 - U.S. OPEN CHAMPIONSHIP...
fetching: June 15, 1999 - U.S. OPEN CHAMPIONSHIP...
fetching: June 6, 1999 - THE MEMORIAL TOURNAMENT...
fetching: Ju

NameError: name 'final_data' is not defined

In [99]:
r_main = requests.get('http://www.asapsports.com/show_player.php?id=10085')
main_soup = BeautifulSoup(r_main.content, 'html.parser')

#final_data = pd.DataFrame()
all_data = []

for interview in main_soup.findAll('table')[-2].findAll('tr')[:5]:
    date = interview.find('nobr').get_text().replace('[ ', '').replace(' ]', '')
    event = interview.find('a').get_text()
    link = interview.find('a')['href']
    print('fetching: ' + date + ' - ' + event + '...')
    
    # use the link to scrape his interview
    r_sub = requests.get(link)
    sub_soup = BeautifulSoup(r_sub.content, 'html.parser')
    # loop through all elements that contain text: 'TIGER WOODS:'
    all_words = ''
    for answer in sub_soup.findAll(text = re.compile('TIGER WOODS')):
        # we always want this item
        text = answer.lstrip().replace('TIGER WOODS:','').replace(',','').replace('.', ' ')
        all_words += text
        # now we need to sometimes get the next paragraph if Tiger keeps talking
        nextanswer = answer
        # look at items after item containing TIGER WOODS:
        while True:
            nextanswer = nextanswer.nextSibling
            # some interviews are structured such that there are no siblings
            # every answer is within its own tag - this is easy then
            if nextanswer == None:
                break
            # all interviews end with this site logo thing
            # signal to stop if we run into it
            if "FastScripts" in str(nextanswer):
                break
            # if its just line breaks, string will be very short    
            # so we only want to look at the longer strings
            # also, we don't want questions (contain 'Q.')
            if (len(str(nextanswer)) <= 10) and ('Q.' not in str(nextanswer)):
                continue
            # sometimes team partner is in interview, check 4th character
            # if uppercase, it's almost certainly someone's name (char. 1,2 are spaces)
            # Ex: "space, space, M, **A**, T, T, space, K, U, C, H,.."
            if (len(str(nextanswer)) > 10) and ('Q.' not in str(nextanswer)):
                if str(nextanswer)[3].isupper():
                    continue
                # otherwise, it's Tiger continuing to talk
                else:
                    text2 = str(nextanswer).lstrip().replace(',', '').replace('.', ' ')
                    all_words += text2
            else:
                break
    
    # run function to clean text and count unique words
    # another function calculates fraction of positive and negative words
    # data = count_unique_words(all_words)
    new = {}
    new['date'] = date
    new['event'] = event
    new['positive'] = sentiment_analysis(all_words)[1]
    new['negative'] = sentiment_analysis(all_words)[0]
    # finally append this dictionary to the grand list
    all_data.append(new)
    
    # add columns to data frame
    #data['date'] = date
    #data['event'] = event
    
    #final_data = final_data.append(data)
    
print('Done!')

#print(final_data)
#write to csv
#final_data.to_csv('~/Dropbox/tiger_interviews', index= False)
all_data = pd.DataFrame(all_data)




fetching: February 2, 2017 - OMEGA DUBAI DESERT CLASSIC...
positive word:well
positive word:well
positive word:just
positive word:pretty
negative word:pain
positive word:just
negative word:trying
positive word:good
positive word:good
negative word:damage
positive word:top
positive word:even
positive word:just
negative word:hard
negative word:little
negative word:little
negative word:ill
negative word:trying
negative word:little
negative word:hole
positive word:well
positive word:hopefully
positive word:good
positive word:even
positive word:hopefully
positive word:solid
negative word:mean
negative word:need
positive word:better
positive word:right
positive word:drive
negative word:need
negative word:hell
positive word:hopefully
positive word:patient
negative word:little
negative word:trying
positive word:even
positive word:even
positive word:just
positive word:even
positive word:moving
positive word:just
positive word:well
positive word:well
positive word:just
positive word:pretty
negat

positive word:pride
positive word:pride
positive word:pride
negative word:trying
positive word:help
positive word:hope
positive word:pride
positive word:pride
positive word:pride
negative word:trying
positive word:help
positive word:hope
fetching: August 23, 2015 - WYNDHAM CHAMPIONSHIP...
negative word:obviously
positive word:might
positive word:well
negative word:little
negative word:difficult
positive word:just
negative word:rough
negative word:hard
positive word:solid
positive word:right
negative word:hole
positive word:good
negative word:mean
positive word:enjoyable
positive word:just
positive word:incredible
positive word:support
positive word:special
positive word:love
positive word:opportunity
positive word:just
positive word:able
positive word:kind
positive word:nice
positive word:pretty
positive word:simple
negative word:wedge
positive word:just
positive word:just
positive word:nice
positive word:sure
positive word:right
positive word:right
positive word:well
positive word:wil

In [95]:
print(sub_soup.p.string)

LEE PATTERSON: All right, sir. Very good day. Maybe just a couple comments about your
round and heading into the weekend.


In [88]:
## just test on any single link here

link = 'http://www.asapsports.com/show_interview.php?id=16031'
#date = interview.find('nobr').get_text().replace('[ ', '').replace(' ]', '')
#event = interview.find('a').get_text()
#link = interview.find('a')['href']
#print('fetching: ' + date + ' - ' + event + '...')
    
# use the link to scrape his interview
r_sub = requests.get(link)
sub_soup = BeautifulSoup(r_sub.content, 'html.parser')
# loop through all elements that contain text: 'TIGER WOODS:'
all_words = ''
for answer in sub_soup.findAll(text = re.compile('TIGER WOODS')):
    # we always want this item
    text = answer.lstrip().replace('TIGER WOODS:','').replace(',','').replace('.', ' ')
    all_words += text
    nextanswer = answer
    # now look at items after item containing TIGER WOODS:
    while True:
        nextanswer = nextanswer.nextSibling
        print(nextanswer)
        if nextanswer == None:
            text3 = nextanswer.find_next('p').get_text()
            if ('Q.' not in text3) and (':' not in text3):
                all_words += text3.lstrip().replace(',', '').replace('.', ' ')
            break
        # all interviews end with this site logo thing
        if "FastScripts" in str(nextanswer):
            break
        if (len(str(nextanswer)) <= 10) and ('Q.' not in str(nextanswer)):
            continue
        if (len(str(nextanswer)) > 10) and ('Q.' not in str(nextanswer)):
            if str(nextanswer)[3].isupper():
                continue
            else:
                text2 = str(nextanswer).lstrip().replace(',', '').replace('.', ' ')
                all_words += text2
        else:
            break
            
        
        # if Tiger is doing a team interview, someone else can answer before next Q.
        # break new answer into words, see if first word is all upper case
        #if str(nextanswer)[2].isupper():
        #        continue
        #we want to keep looking at the next.sibling until we hit a Q.
        #if "Q." not in str(nextanswer):
        #    text2 = str(nextanswer).lstrip().replace(',', '').replace('.', ' ')
        #    all_words += text2
        #else:
            #break
# run function to clean text and count unique words
#data = count_unique_words(all_words)
# add columns to data frame
#data['date'] = date
#data['event'] = event
    
print(all_words)

<a name="end1"><p>
<b><a name="q2">Q.  (Inaudible).<a name="endq2"></a></a></b><p>
<a name="2">TIGER WOODS:  You know, in a sense it is, it really is, because being away and just having so much fun out there, just being by ourselves and diving and just having a great time, it was just something I haven't been able to do that much this year, and certainly all the years I've been on Tour, I've done that, but this year I haven't taken as much time off, so it was nice actually to shut it down for a little bit.<a name="end2"><p>
<b><a name="q3">Q.  (Inaudible).<a name="endq3"></a></a></b><p>
<a name="3">TIGER WOODS:  Almost three weeks.<a name="end3"><p>
<b><a name="q4">Q.  (Inaudible).<a name="endq4"></a></a></b><p>
<a name="4">TIGER WOODS:  Well, I needed to because my rib was not very good.  I couldn't really do much.  I don't know how I got through Amex, and then it took me another two weeks after that before it actually settled down, so it took me a while.<a name="end4"><p>
<b><a name=

In [43]:
link = 'http://www.asapsports.com/show_interview.php?id=15040'
#date = interview.find('nobr').get_text().replace('[ ', '').replace(' ]', '')
#event = interview.find('a').get_text()
#link = interview.find('a')['href']
#print('fetching: ' + date + ' - ' + event + '...')
    
# use the link to scrape his interview
r_sub = requests.get(link)
sub_soup = BeautifulSoup(r_sub.content, 'html.parser')
# loop through all elements that contain text: 'TIGER WOODS:'
all_words = ''
for answer in sub_soup.findAll(text = re.compile('TIGER WOODS')):
    # we always want this item
    print(answer)
    text3 = answer.findNext('p').get_text()
    print(text3)
    print(('Q.' not in text3) and (':' not in text3))
    break
      
            
        

TIGER WOODS:  You can put it this way:  I've said this before when I was asked out on the boat, just because you're married doesn't change your relationship.  It changes people's perception of the relationship, but the way Elin and I feel about each other doesn't change at all, just because we've taken another step.  I've felt this way about her for a long time, and nothing is going to change that.

Q.  (Inaudible).
TIGER WOODS:  You know, in a sense it is, it really is, because being away and just having so much fun out there, just being by ourselves and diving and just having a great time, it was just something I haven't been able to do that much this year, and certainly all the years I've been on Tour, I've done that, but this year I haven't taken as much time off, so it was nice actually to shut it down for a little bit.
Q.  (Inaudible).
TIGER WOODS:  Almost three weeks.
Q.  (Inaudible).
TIGER WOODS:  Well, I needed to because my rib was not very good.  I couldn't really do much.  

In [23]:
r_main = requests.get('http://www.asapsports.com/show_player.php?id=10085')
main_soup = BeautifulSoup(r_main.content, 'html.parser')

all_data = []

# loop through all the interviews (only does 5 for testing)
for interview in main_soup.findAll('table')[-2].findAll('tr')[:5]: # this is only going to do the first 5!!
    # get the date, event, and link
    date = interview.find('nobr').get_text().replace('[ ', '').replace(' ]', '')
    event = interview.find('a').get_text()
    link = interview.find('a')['href']
    print('fetching: ' + date + ' - ' + event + '...')
    
    # use the link to scrape his interview
    r_sub = requests.get(link)
    sub_soup = BeautifulSoup(r_sub.content, 'html.parser')
    # loop through all elements that contain text: 'TIGER WOODS:'
    for answer in sub_soup.findAll(text = re.compile('TIGER WOODS:')):
        new = {}
        # get text and replace his name, comma, period
        text = answer.lstrip().replace('TIGER WOODS: ', '').replace(',', '').replace('.', '')
        # split text into python list of words
        words = text.split()
        
        ####################################
        # DO SHIT HERE TO THE LIST OF WORDS
        ####################################
        
        # fill in dictionary to complete observation
        new['date'] = date
        new['event'] = event
        new['text'] = words
        # finally append this dictionary to the grand list
        all_data.append(new)

print('Done!!')
# turn this list of dictionaries into a pandas df
#all_data = pd.DataFrame(all_data)

fetching: February 2, 2017 - OMEGA DUBAI DESERT CLASSIC...
fetching: February 1, 2017 - OMEGA DUBAI DESERT CLASSIC...
fetching: October 2, 2016 - THE RYDER CUP...
fetching: August 23, 2015 - WYNDHAM CHAMPIONSHIP...
fetching: August 22, 2015 - WYNDHAM CHAMPIONSHIP...
Done!!
